In [21]:
import argparse
from pathlib import Path
import pandas as pd
import torch
import json
import random
import numpy as np
from typing import List, Tuple
import tiktoken
from dataclasses import dataclass
import os
import time
from pymatgen.core.composition import Composition
from pymatgen.core.structure import Structure


In [22]:
def contains_elements(comp, target_elements):
    """Check if composition contains all target elements (Ag, O) regardless of ratio."""
    return all(el in comp.elements for el in target_elements)

def matches_composition(comp, target_elements, target_ratio):
    """Check if composition contains all target elements (Ag, O) regardless of ratio."""
    if not all(el in comp.elements for el in target_elements):
        return False
    reduced_comp, _ = comp.get_reduced_composition_and_factor()
    return all(abs(reduced_comp[el] - amt) <= 1e-6 for el, amt in target_ratio.items())


In [ ]:
seed_structures_df = pd.read_csv('resources/band_gap_processed.csv')
seed_structures_df['structure'] = seed_structures_df['structure'].apply(lambda x: Structure.from_str(x, fmt='json') if pd.notna(x) else None)
seed_structures_df['composition'] = [s.composition for s in seed_structures_df['structure']]
seed_structures_df['composition_str'] = [s.composition.formula for s in seed_structures_df['structure']]

In [ ]:
comps = ['Ag6O2', 'Bi2F8', 'Co2Sb2', 'Co4B2', 'Cr4Si4', 'KZnF3', "Sr2O4", "YMg3"]
for comp in comps:
    target_comp = Composition(comp)
    target_elements = set(target_comp.elements)
    comp_df = seed_structures_df[seed_structures_df['composition'].apply(lambda comp: contains_elements(comp, target_elements))]
    print(comp, len(comp_df))

In [23]:
import importlib
import utils
importlib.reload(utils.stability)
from utils.stability import StabilityCalculator
# from utils.e_hull_calculator import EHullCalculator
# ehull_model = EHullCalculator('resources/2023-02-07-ppd-mp.pkl.gz')
stability_calculator_chgnet = StabilityCalculator(mlip='chgnet', ppd_path='resources/2023-02-07-ppd-mp.pkl.gz')
stability_calculator_orbv3 = StabilityCalculator(mlip='orb-v3', ppd_path='resources/2023-02-07-ppd-mp.pkl.gz')
stability_calculator_sevenet = StabilityCalculator(mlip='sevenet', ppd_path='resources/2023-02-07-ppd-mp.pkl.gz')

Initialize EHullCalcul with patched phase diagram.
CHGNet initialized with 412,525 parameters
CHGNet initialized with 412,525 parameters
CHGNet will run on cuda:7
Initialize EHullCalcul with patched phase diagram.


/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


Initialize EHullCalcul with patched phase diagram.


In [12]:
save_label = "poscar_70b_csp_KZnF3"
generations_df = pd.read_csv(f'../results/{save_label}/generations.csv')
generations_df.head()


,Iteration,Structure,ParentStructures,Objective,Composition,DeltaE,EHullDistance,BulkModulus,StructureRelaxed,BulkModulusRelaxed
0,1,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...","[""{\""@class\"": \""Structure\"", \""@module\"": \""p...",inf,F3 K1 Zn1,-1.874561,0.795208,0.0,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...",0.0
1,1,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...","[""{\""@class\"": \""Structure\"", \""@module\"": \""p...",inf,F3 K1 Zn1,-0.917131,0.373527,0.0,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...",0.0
2,1,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...","[""{\""@class\"": \""Structure\"", \""@module\"": \""p...",inf,F3 K1 Zn1,0.000000,1.717362,0.0,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...",0.0
3,1,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...","[""{\""@class\"": \""Structure\"", \""@module\"": \""p...",inf,F3 Zn1 K1,-1.814619,0.661457,0.0,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...",0.0
4,1,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...","[""{\""@class\"": \""Structure\"", \""@module\"": \""p...",inf,K1 Zn1 F3,-2.028812,0.581382,0.0,"{""@class"": ""Structure"", ""@module"": ""pymatgen.c...",0.0


In [24]:
structures = generations_df['Structure'].apply(lambda x: Structure.from_str(x, fmt='json') if pd.notna(x) else None)
stability_results = structures.apply(lambda x: stability_calculator_orbv3.compute_stability([x])[0])
generations_df['EHullDistance_orbv3'] = stability_results.apply(lambda x: x.e_hull_distance if x is not None else None)


generations_df['StructureRelaxed'] = stability_results.apply(lambda x: json.dumps(x.structure_relaxed.as_dict(), sort_keys=True) if x is not None else None)
generations_df['EHullDistance'] = stability_results.apply(lambda x: x.e_hull_distance if x is not None else None)
generations_df['DeltaE'] = stability_results.apply(lambda x: x.delta_e if x is not None else None)

      Step     Time          Energy          fmax
BFGS:    0 11:54:33       -8.846066       13.070996
BFGS:    1 11:54:33      -14.306476        6.143689
BFGS:    2 11:54:33      -15.912448        4.652773
BFGS:    3 11:54:33      -17.268463        3.121026
BFGS:    4 11:54:33      -17.651569        2.886102
BFGS:    5 11:54:33      -17.830595        1.463712
BFGS:    6 11:54:33      -17.937727        0.783484
BFGS:    7 11:54:33      -18.010599        1.102025
BFGS:    8 11:54:33      -18.056536        1.163587
BFGS:    9 11:54:33      -18.129848        0.940910
BFGS:   10 11:54:33      -18.193956        0.669462
BFGS:   11 11:54:33      -18.217676        0.160109
BFGS:   12 11:54:33      -18.218336        0.085839
BFGS:   13 11:54:33      -18.217726        0.058774
BFGS:   14 11:54:33      -18.218237        0.021784


  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1037.17it/s]

      Step     Time          Energy          fmax
BFGS:    0 11:54:33      -15.741152        1.973118
BFGS:    1 11:54:33      -15.848158        1.865280
BFGS:    2 11:54:33      -16.188684        1.911899
BFGS:    3 11:54:33      -16.320507        2.015089
BFGS:    4 11:54:33      -16.489311        1.014830


BFGS:    5 11:54:33      -16.514492        0.396596
BFGS:    6 11:54:33      -16.526707        0.567617
BFGS:    7 11:54:34      -16.554239        0.842156
BFGS:    8 11:54:34      -16.583815        0.801781
BFGS:    9 11:54:34      -16.606270        0.418201
BFGS:   10 11:54:34      -16.616699        0.368195
BFGS:   11 11:54:34      -16.622097        0.333387
BFGS:   12 11:54:34      -16.633966        0.651743
BFGS:   13 11:54:34      -16.682774        1.664584
BFGS:   14 11:54:34      -16.688799        3.041218
BFGS:   15 11:54:34      -16.773438        2.367262
BFGS:   16 11:54:34      -16.879940        2.648983
BFGS:   17 11:54:34      -17.733322        2.560463
BFGS:   18 11:54:34      -18.301918        1.384865
BFGS:   19 11:54:34      -18.382389        2.823224
BFGS:   20 11:54:34      -18.558659        1.126208
BFGS:   21 11:54:34      -18.655571        1.150024
BFGS:   22 11:54:34      -18.872202        1.804966
BFGS:   23 11:54:34      -19.083904        2.434276
BFGS:   24 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4675.92it/s]

...

BFGS:    5 11:57:23      -14.510540        3.763445
BFGS:    6 11:57:23      -15.159951        2.674996
BFGS:    7 11:57:23      -15.468372        3.212623
BFGS:    8 11:57:23      -15.975377        3.857772
BFGS:    9 11:57:23      -16.569632        3.693774
BFGS:   10 11:57:23      -17.173967        4.640920
BFGS:   11 11:57:23      -17.901339        3.770126
BFGS:   12 11:57:23      -18.290384        1.555230
BFGS:   13 11:57:23      -18.388216        1.256829
BFGS:   14 11:57:23      -18.450085        0.915531
BFGS:   15 11:57:23      -18.500450        0.699630
BFGS:   16 11:57:24      -18.549618        0.553271
BFGS:   17 11:57:24      -18.557325        0.366355
BFGS:   18 11:57:24      -18.562084        0.267909
BFGS:   19 11:57:24      -18.566250        0.230225
BFGS:   20 11:57:24      -18.571039        0.200635
BFGS:   21 11:57:24      -18.575632        0.152660
BFGS:   22 11:57:24      -18.576757        0.063393
BFGS:   23 11:57:24      -18.576889        0.036701


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4120.14it/s]

      Step     Time          Energy          fmax
BFGS:    0 11:57:24        0.070429       25.416104
BFGS:    1 11:57:24       -4.407478        9.540350
BFGS:    2 11:57:24       -5.724586        3.332364
BFGS:    3 11:57:24       -6.821964        3.020399
BFGS:    4 11:57:24       -8.145231        3.223014
BFGS:    5 11:57:24       -9.214771        4.129383


BFGS:    6 11:57:24      -10.093929        4.779156
BFGS:    7 11:57:24      -10.887297        5.448815
BFGS:    8 11:57:24      -11.678387        6.199771
BFGS:    9 11:57:24      -12.804624        4.421925
BFGS:   10 11:57:24      -14.289339        3.495354
BFGS:   11 11:57:24      -15.405827        3.626585
BFGS:   12 11:57:24      -16.159739        2.967785
BFGS:   13 11:57:25      -16.403318        3.208175
BFGS:   14 11:57:25      -16.601969        2.627596
BFGS:   15 11:57:25      -17.457085        1.987621
BFGS:   16 11:57:25      -18.089005        1.583427
BFGS:   17 11:57:25      -18.595619        1.441081
BFGS:   18 11:57:25      -19.003601        1.673700
BFGS:   19 11:57:25      -19.261524        1.721534
BFGS:   20 11:57:25      -19.398548        1.928002
BFGS:   21 11:57:25      -19.739670        1.845107
BFGS:   22 11:57:25      -19.934769        1.105469
BFGS:   23 11:57:25      -20.014011        0.776382
BFGS:   24 11:57:25      -20.056900        0.781765
BFGS:   25 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4793.49it/s]


In [25]:
structures = generations_df['Structure'].apply(lambda x: Structure.from_str(x, fmt='json') if pd.notna(x) else None)
stability_results = structures.apply(lambda x: stability_calculator_chgnet.compute_stability([x])[0])
generations_df['EHullDistance_chgnet'] = stability_results.apply(lambda x: x.e_hull_distance if x is not None else None)


      Step     Time          Energy          fmax
FIRE:    0 11:57:25       -9.371120       49.998820
FIRE:    1 11:57:25      -17.735864       10.388581


/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/chgnet/model/dynamics.py:237: FutureWarning: Import ExpCellFilter from ase.filters
  atoms = ExpCellFilter(atoms)


FIRE:    2 11:57:26      -18.548434        7.248168
FIRE:    3 11:57:26      -18.765268        6.797432
FIRE:    4 11:57:26      -19.144398        4.841592
FIRE:    5 11:57:26      -19.509670        2.571663
FIRE:    6 11:57:26      -19.706053        3.130667
FIRE:    7 11:57:26      -19.767042        7.104988
FIRE:    8 11:57:26      -19.819505        6.479119
FIRE:    9 11:57:26      -19.907414        5.249054
FIRE:   10 11:57:26      -19.996613        3.682853
FIRE:   11 11:57:26      -20.077350        2.006228
FIRE:   12 11:57:26      -20.132751        1.671085
FIRE:   13 11:57:26      -20.173264        1.652909
FIRE:   14 11:57:26      -20.204470        1.655223
FIRE:   15 11:57:26      -20.245183        1.841159
FIRE:   16 11:57:26      -20.312364        1.968112
FIRE:   17 11:57:27      -20.396273        1.787795
FIRE:   18 11:57:27      -20.507550        1.670894
FIRE:   19 11:57:27      -20.615795        1.468115
FIRE:   20 11:57:27      -20.703831        1.348435
FIRE:   21 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2438.55it/s]

      Step     Time          Energy          fmax
FIRE:    0 11:57:29      -17.975179       12.017547
FIRE:    1 11:57:29      -19.339509        6.354364
FIRE:    2 11:57:29      -20.367436        3.075199


...

FIRE:    2 12:05:57      -18.211081        7.211534
FIRE:    3 12:05:57      -18.515527        6.259931
FIRE:    4 12:05:57      -19.008982        4.220977
FIRE:    5 12:05:57      -19.487039        4.121707
FIRE:    6 12:05:57      -19.722078        7.221711
FIRE:    7 12:05:57      -19.771061        6.775216
FIRE:    8 12:05:57      -19.849340        5.939926
FIRE:    9 12:05:57      -19.945428        4.861667
FIRE:   10 12:05:57      -20.036681        3.822600
FIRE:   11 12:05:57      -20.116947        2.804893
FIRE:   12 12:05:57      -20.189588        2.144532
FIRE:   13 12:05:57      -20.260041        2.013541
FIRE:   14 12:05:58      -20.334828        2.060231
FIRE:   15 12:05:58      -20.426128        1.771014
FIRE:   16 12:05:58      -20.522735        1.440442
FIRE:   17 12:05:58      -20.604777        1.344417
FIRE:   18 12:05:58      -20.658324        1.328508
FIRE:   19 12:05:58      -20.693033        1.727242
FIRE:   20 12:05:58      -20.745556        1.198712
FIRE:   21 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 3905.31it/s]

      Step     Time          Energy          fmax
FIRE:    0 12:06:03        1.928339       64.907873
FIRE:    1 12:06:03      -11.436660       28.626764


FIRE:    2 12:06:03      -14.787886        8.487948
FIRE:    3 12:06:03      -14.999179        6.755781
FIRE:    4 12:06:03      -15.229081        6.384735
FIRE:    5 12:06:03      -15.665557        6.194602
FIRE:    6 12:06:04      -16.228604        4.968612
FIRE:    7 12:06:04      -16.707817        3.134505
FIRE:    8 12:06:04      -16.968603        4.605167
FIRE:    9 12:06:04      -17.200251        6.380216
FIRE:   10 12:06:04      -17.592866        6.185523
FIRE:   11 12:06:04      -18.129448        4.524948
FIRE:   12 12:06:04      -18.533099        2.936487
FIRE:   13 12:06:04      -18.667418        2.972552
FIRE:   14 12:06:04      -18.702070        2.912504
FIRE:   15 12:06:04      -18.768995        2.831399
FIRE:   16 12:06:04      -18.856325        2.733917
FIRE:   17 12:06:04      -18.947427        2.300010
FIRE:   18 12:06:04      -19.025836        2.127947
FIRE:   19 12:06:04      -19.071782        2.346465
FIRE:   20 12:06:05      -19.092733        3.060167
FIRE:   21 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 3412.78it/s]


In [26]:
structure = structures[0]
relaxation = stability_calculator_chgnet.relax_structure(structure)
print(relaxation['trajectory'])

      Step     Time          Energy          fmax
FIRE:    0 12:06:16       -9.371120       49.998820
FIRE:    1 12:06:16      -17.735864       10.390267
FIRE:    2 12:06:16      -18.548126        7.243880
FIRE:    3 12:06:16      -18.765082        6.797487
FIRE:    4 12:06:16      -19.143770        4.846093
FIRE:    5 12:06:16      -19.510677        2.571346
FIRE:    6 12:06:16      -19.705166        3.128725
FIRE:    7 12:06:16      -19.769564        7.104663
FIRE:    8 12:06:16      -19.821253        6.475167
FIRE:    9 12:06:16      -19.900728        5.313260
FIRE:   10 12:06:16      -19.997710        3.662552
FIRE:   11 12:06:16      -20.078502        1.957649
FIRE:   12 12:06:16      -20.132008        1.668468
FIRE:   13 12:06:16      -20.172584        1.653523
FIRE:   14 12:06:16      -20.204318        1.655094
FIRE:   15 12:06:17      -20.245504        1.839548
FIRE:   16 12:06:17      -20.312665        1.968319
FIRE:   17 12:06:17      -20.400438        1.779556
FIRE:   18 12:

In [ ]:
structures = generations_df['Structure'].apply(lambda x: Structure.from_str(x, fmt='json') if pd.notna(x) else None)
stability_results = structures.apply(lambda x: stability_calculator_sevenet.compute_stability([x])[0])
generations_df['EHullDistance_sevenet'] = stability_results.apply(lambda x: x.e_hull_distance if x is not None else None)


      Step     Time          Energy          fmax
BFGS:    0 12:06:23       -9.790884       13.945059
BFGS:    1 12:06:24      -15.393970        5.386542
BFGS:    2 12:06:24      -16.615273        3.510282
BFGS:    3 12:06:24      -17.996046        3.571234
BFGS:    4 12:06:25      -18.332445        3.029580
BFGS:    5 12:06:25      -18.530822        1.719933
BFGS:    6 12:06:25      -18.628676        1.056936
BFGS:    7 12:06:25      -18.742669        1.147391
BFGS:    8 12:06:25      -18.786713        1.263222
BFGS:    9 12:06:25      -18.861390        1.119646
BFGS:   10 12:06:26      -18.930797        0.658447
BFGS:   11 12:06:26      -18.958703        0.134384
BFGS:   12 12:06:26      -18.958802        0.029231


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2492.16it/s]


      Step     Time          Energy          fmax
BFGS:    0 12:06:26      -16.484535        1.935391
BFGS:    1 12:06:27      -16.590079        1.843226
BFGS:    2 12:06:27      -16.891430        1.876502
BFGS:    3 12:06:27      -17.028083        2.094023
BFGS:    4 12:06:27      -17.230831        1.196600
BFGS:    5 12:06:28      -17.264640        0.484086
BFGS:    6 12:06:28      -17.274497        0.512356
BFGS:    7 12:06:28      -17.294224        0.782165
BFGS:    8 12:06:28      -17.320535        0.860377
BFGS:    9 12:06:28      -17.346662        0.544197
BFGS:   10 12:06:29      -17.356359        0.301929
BFGS:   11 12:06:29      -17.360111        0.281027
BFGS:   12 12:06:29      -17.364100        0.220876
BFGS:   13 12:06:29      -17.368441        0.265261
BFGS:   14 12:06:29      -17.372156        0.194618
BFGS:   15 12:06:30      -17.373874        0.061516
BFGS:   16 12:06:30      -17.373769        0.007119


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5817.34it/s]


      Step     Time          Energy          fmax
BFGS:    0 12:06:31      -14.145348        0.000000


...

      Step     Time          Energy          fmax
BFGS:    0 12:17:14      -12.313373        3.720436
BFGS:    1 12:17:15      -12.677023        3.950017
BFGS:    2 12:17:15      -14.076296        3.137937
BFGS:    3 12:17:15      -13.942096        8.134875
BFGS:    4 12:17:15      -14.433860        1.531855
BFGS:    5 12:17:15      -14.535021        1.063777
BFGS:    6 12:17:15      -14.668853        2.105502
BFGS:    7 12:17:16      -14.745230        1.784946
BFGS:    8 12:17:16      -14.942989        1.572240
BFGS:    9 12:17:16      -15.038642        1.649685
BFGS:   10 12:17:16      -15.328576        1.611386
BFGS:   11 12:17:16      -15.568834        2.271707
BFGS:   12 12:17:17      -15.698092        2.618411
BFGS:   13 12:17:17      -15.748868        2.602197
BFGS:   14 12:17:17      -16.074491        1.947030
BFGS:   15 12:17:17      -16.308328        1.397303
BFGS:   16 12:17:17      -16.358582        1.290567
BFGS:   17 12:17:17      -16.512765        1.144785
BFGS:   18 12:

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2811.20it/s]


      Step     Time          Energy          fmax
BFGS:    0 12:17:32       -9.857223       10.908127
BFGS:    1 12:17:33      -12.695236        4.550127
BFGS:    2 12:17:33      -13.530799        3.839953
BFGS:    3 12:17:33      -14.327459        4.607915
BFGS:    4 12:17:33      -14.859953        4.797371
BFGS:    5 12:17:33      -15.326769        4.069030
BFGS:    6 12:17:34      -16.109240        2.699151
BFGS:    7 12:17:34      -16.459964        3.359298
BFGS:    8 12:17:34      -16.980653        3.825885
BFGS:    9 12:17:34      -17.557306        3.659869
BFGS:   10 12:17:34      -18.274812        3.851335
BFGS:   11 12:17:35      -18.842520        2.262963
BFGS:   12 12:17:35      -18.991157        1.341546
BFGS:   13 12:17:35      -19.112213        1.353735
BFGS:   14 12:17:35      -19.191807        1.254121
BFGS:   15 12:17:36      -19.259990        0.694919
BFGS:   16 12:17:36      -19.279910        0.369227
BFGS:   17 12:17:36      -19.285798        0.280796
BFGS:   18 12:

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4471.54it/s]


      Step     Time          Energy          fmax
BFGS:    0 12:17:37       -0.613948       24.988809
BFGS:    1 12:17:38       -4.767301       11.665973
BFGS:    2 12:17:38       -6.427029        3.839926
BFGS:    3 12:17:38       -7.747810        3.563811
BFGS:    4 12:17:38       -9.109450        4.333477
BFGS:    5 12:17:38      -10.046427        5.014699
BFGS:    6 12:17:39      -10.772766        4.447969
BFGS:    7 12:17:39      -11.471727        3.839034
BFGS:    8 12:17:39      -12.231466        4.848905
BFGS:    9 12:17:39      -13.326107        6.074354
BFGS:   10 12:17:40      -14.832350        5.589120
BFGS:   11 12:17:40      -15.780969        5.614847
BFGS:   12 12:17:40      -16.348270        4.948669
BFGS:   13 12:17:40      -16.990263        4.531244
BFGS:   14 12:17:41      -17.774450        3.118765
BFGS:   15 12:17:41      -18.109864        2.602305
BFGS:   16 12:17:41      -18.270291        2.713839
BFGS:   17 12:17:41      -18.761032        2.748057
BFGS:   18 12:

In [ ]:
generations_df.to_csv(f'../results/{save_label}/generations_compare.csv', index=False)

In [ ]:
stats_df = pd.DataFrame({
    'mean': [generations_df['EHullDistance_sevenet'].mean(),
             generations_df['EHullDistance_chgnet'].mean(),
             generations_df['EHullDistance_orbv3'].mean()],
    'min': [generations_df['EHullDistance_sevenet'].min(),
            generations_df['EHullDistance_chgnet'].min(),
            generations_df['EHullDistance_orbv3'].min()],
    'max': [generations_df['EHullDistance_sevenet'].max(),
            generations_df['EHullDistance_chgnet'].max(),
            generations_df['EHullDistance_orbv3'].max()]
})
stats_df.index = ['sevenet', 'chgnet', 'orbv3']

# Print statistics
print("Statistics for EHull Distance values:")
print(stats_df)

# Show a few examples
print("\nExample values (first 5 rows):")
examples = generations_df[['EHullDistance_sevenet', 
                          'EHullDistance_chgnet', 
                          'EHullDistance_orbv3']].head()
print(examples)

In [60]:
generations_df = pd.read_csv(f'../results/{save_label}/generations.csv')
generations_df['EHullDistance']-generations_df_['EHullDistance']
# generations_df['DeltaE']-generations_df_['DeltaE']

0     0.633691
1     0.277096
2     1.561936
3     0.530927
4     0.364662
        ...   
70    0.335799
71    0.416198
72    0.661585
73    0.633039
74    0.363765
Name: EHullDistance, Length: 75, dtype: float64

In [59]:
structure = structures[6]
energy = stability_calculator_chgnet.compute_energy_per_atom(structure)
print('energy chgnet', energy)
energy_ = stability_calculator_orbv3.compute_energy_per_atom(structure)
print('energy orbv3', energy_)
relaxation = stability_calculator_chgnet.relax_structure(structure)
print('relaxation', relaxation)
relaxation_ = stability_calculator_orbv3.relax_structure(structure)
print('relaxation', relaxation_)
structure_relaxed = relaxation['final_structure']
structure_relaxed_ = relaxation_['final_structure']
energy_relaxed = stability_calculator_chgnet.compute_energy_per_atom(structure_relaxed)
print('energy_relaxed', energy_relaxed)
energy_relaxed_ = stability_calculator_orbv3.compute_energy_per_atom(structure_relaxed_)
print('energy_relaxed', energy_relaxed_)

e_hull_distance = stability_calculator_chgnet.compute_ehull_dist(structure_relaxed, energy_relaxed) 
e_hull_distance_ = stability_calculator_orbv3.compute_ehull_dist(structure_relaxed_, energy_relaxed_) 
print('e_hull_distance', e_hull_distance, e_hull_distance_)

energy chgnet -2.0863840579986572
energy orbv3 -1.7150611877441406
      Step     Time          Energy          fmax
FIRE:    0 23:51:37      -10.429727       24.679861
FIRE:    1 23:51:37      -14.791532        4.302103
FIRE:    2 23:51:37      -14.962103        5.026131


/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/chgnet/model/dynamics.py:237: FutureWarning: Import ExpCellFilter from ase.filters
  atoms = ExpCellFilter(atoms)


FIRE:    3 23:51:37      -15.100472        4.801128
FIRE:    4 23:51:37      -15.380871        4.861904
FIRE:    5 23:51:37      -15.813762        4.482694
FIRE:    6 23:51:37      -16.275381        3.753484
FIRE:    7 23:51:37      -16.670322        3.221899
FIRE:    8 23:51:37      -16.964349        4.696006
FIRE:    9 23:51:37      -17.337703        4.891908
FIRE:   10 23:51:37      -17.876228        5.005354
FIRE:   11 23:51:37      -18.457884        4.042971
FIRE:   12 23:51:37      -18.943471        2.843278
FIRE:   13 23:51:38      -19.383278        2.758730
FIRE:   14 23:51:38      -19.789118        2.557762
FIRE:   15 23:51:38      -20.109429        2.808764
FIRE:   16 23:51:38      -20.433445        2.635607
FIRE:   17 23:51:38      -20.661502        3.393600
FIRE:   18 23:51:38      -20.743437        2.083626
FIRE:   19 23:51:38      -20.802956        1.515449
FIRE:   20 23:51:38      -20.811725        1.386216
FIRE:   21 23:51:38      -20.825481        1.190453
FIRE:   22 2

  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4429.04it/s]

e_hull_distance 0.11256942460009434 0.41828365036913695


In [10]:
stability_results = seed_structures_df[:3]['structure'].apply(lambda x: stability_calculator_chgnet.compute_stability([x])[0])


/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/chgnet/model/model.py:759: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /pytorch/aten/src/ATen/native/Cross.cpp:62.)
  volumes.append(torch.dot(lattice[0], torch.cross(lattice[1], lattice[2])))
/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/chgnet/model/dynamics.py:237: FutureWarning: Import ExpCellFilter from ase.filters
  atoms = ExpCellFilter(atoms)


      Step     Time          Energy          fmax
FIRE:    0 22:28:01     -109.424622        0.758916
FIRE:    1 22:28:01     -109.432205        0.717667
FIRE:    2 22:28:01     -109.416626        2.688207
FIRE:    3 22:28:01     -109.429115        1.080351
FIRE:    4 22:28:01     -109.434593        0.791901
FIRE:    5 22:28:01     -109.431854        0.460174
FIRE:    6 22:28:01     -109.433754        0.456066
FIRE:    7 22:28:01     -109.437897        0.389377
FIRE:    8 22:28:01     -109.436249        0.354471
FIRE:    9 22:28:01     -109.434708        0.334662
FIRE:   10 22:28:01     -109.435349        0.294138
FIRE:   11 22:28:02     -109.438332        0.160065
FIRE:   12 22:28:02     -109.431656        0.200378
FIRE:   13 22:28:02     -109.432053        0.174881
FIRE:   14 22:28:02     -109.433456        0.214812
FIRE:   15 22:28:02     -109.434082        0.235061
FIRE:   16 22:28:02     -109.434731        0.236065
FIRE:   17 22:28:02     -109.438965        0.201615
FIRE:   18 22:

  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]/local2/jrgan/anaconda3/envs/matllmsearch/lib/python3.10/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]

E-hull computation error: Unable to get decomposition for None ComputedStructureEntry - K4 Mn4 O8    (KMnO2)
Energy (Uncorrected)     = -109.4375 eV (-6.8398  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -109.4375 eV (-6.8398  eV/atom)
Energy Adjustments:
  None
Parameters:
Data:
      Step     Time          Energy          fmax
FIRE:    0 22:28:03      -33.942574        0.540111
FIRE:    1 22:28:03      -33.939793        0.549961


FIRE:    2 22:28:03      -33.940987        0.298507
FIRE:    3 22:28:03      -33.941803        0.097068


  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]

E-hull computation error: Unable to get decomposition for None ComputedStructureEntry - Cr3 Ni1      (Cr3Ni)
Energy (Uncorrected)     = -33.9413  eV (-8.4853  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -33.9413  eV (-8.4853  eV/atom)
Energy Adjustments:
  None
Parameters:
Data:
      Step     Time          Energy          fmax
FIRE:    0 22:28:03       -7.270300        0.518828
FIRE:    1 22:28:03       -7.277307        0.383347
FIRE:    2 22:28:03       -7.280029        0.115481
FIRE:    3 22:28:03       -7.280678        0.238777
FIRE:    4 22:28:03       -7.281534        0.208204


FIRE:    5 22:28:03       -7.281169        0.174603
FIRE:    6 22:28:03       -7.281131        0.119441
FIRE:    7 22:28:03       -7.282486        0.069151


  0%|                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]

E-hull computation error: Unable to get decomposition for None ComputedStructureEntry - Cs1 Rb1 As1  (CsRbAs)
Energy (Uncorrected)     = -7.2821   eV (-2.4274  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -7.2821   eV (-2.4274  eV/atom)
Energy Adjustments:
  None
Parameters:
Data:


In [62]:
import os
import pandas as pd
from pymatgen.io.cif import CifWriter

output_dir = "visualization"
os.makedirs(output_dir, exist_ok=True)

# Remove None values and sort by EHullDistance
valid_structures = generations_df.dropna(subset=['EHullDistance'])
sorted_structures = valid_structures.sort_values(by='EHullDistance')

# Define how many of the best structures to save
num_structures = 20  # Adjust this number as needed

# Save top structures to CIF files
for i, (index, row) in enumerate(sorted_structures.head(num_structures).iterrows()):
    structure = Structure.from_str(row['StructureRelaxed'], fmt='json')
    formula = structure.composition.reduced_formula
    e_hull = row['EHullDistance']
    
    # Create a descriptive filename
    filename = f"{i+1:03d}_{formula}_ehull_{e_hull:.4f}.cif"
    filepath = os.path.join("visualization", filename)
    
    # Write the structure to a CIF file
    cif_writer = CifWriter(structure)
    cif_writer.write_file(filepath)
    
    print(f"Saved structure {i+1}: {formula} with E-hull = {e_hull:.4f} eV/atom to {filepath}")


Saved structure 1: KZnF3 with E-hull = 0.3735 eV/atom to visualization/001_KZnF3_ehull_0.3735.cif
Saved structure 2: KZnF3 with E-hull = 0.4183 eV/atom to visualization/002_KZnF3_ehull_0.4183.cif
Saved structure 3: KZnF3 with E-hull = 0.4994 eV/atom to visualization/003_KZnF3_ehull_0.4994.cif
Saved structure 4: KZnF3 with E-hull = 0.5066 eV/atom to visualization/004_KZnF3_ehull_0.5066.cif
Saved structure 5: KZnF3 with E-hull = 0.5172 eV/atom to visualization/005_KZnF3_ehull_0.5172.cif
Saved structure 6: KZnF3 with E-hull = 0.5242 eV/atom to visualization/006_KZnF3_ehull_0.5242.cif
Saved structure 7: KZnF3 with E-hull = 0.5267 eV/atom to visualization/007_KZnF3_ehull_0.5267.cif
Saved structure 8: KZnF3 with E-hull = 0.5814 eV/atom to visualization/008_KZnF3_ehull_0.5814.cif
Saved structure 9: KZnF3 with E-hull = 0.5849 eV/atom to visualization/009_KZnF3_ehull_0.5849.cif
Saved structure 10: KZnF3 with E-hull = 0.6147 eV/atom to visualization/010_KZnF3_ehull_0.6147.cif
Saved structure 11:

In [61]:
def generation_dedup(df) :
    unique_structures = []
    keep_indices = []
    reduced_formulas = []

    for idx, struct_str in enumerate(df['StructureRelaxed']):
        try:
            structure = Structure.from_dict(json.loads(struct_str))
            reduced_formula = structure.composition.reduced_formula
            df.at[idx, 'Composition'] = structure.composition
            df.at[idx, 'composition_str'] = structure.composition.formula
            is_duplicate = any(
                structure.matches(unique_struct, scale=True, attempt_supercell=False)
                for unique_struct, formula in zip(unique_structures, reduced_formulas)
                if formula == reduced_formula
            )
            if not is_duplicate:
                unique_structures.append(structure)
                reduced_formulas.append(reduced_formula)
                keep_indices.append(idx)
            else:
                print(idx, df.at[idx, 'EHullDistance'])
                for j, (unique_struct, formula) in enumerate(zip(unique_structures, reduced_formulas)):
                    if structure.matches(unique_struct, scale=True, attempt_supercell=False):
                        print(idx,j,keep_indices[j])
        except Exception as e:
            print(f"Error processing structure at index {idx}: {e}")
            continue
    return df.iloc[keep_indices].reset_index(drop=True)


def valid_value(x):
    return (x is not None and not np.isinf(x) and not np.isnan(x) and x != 0)


def valid_mean(values):
    valid_values = [x for x in values if valid_value(x)]
    return np.mean(valid_values) if valid_values else 0.0
# generations_df = pd.read_csv(f'../results/{save_label}/generations_chgnet.csv')

generations_df = generation_dedup(generations_df)

7 0.5601926774687467
7 6 6
8 0.4524274797148404
8 1 1
9 0.4070173234648404
9 1 1
9 6 6
10 0.4080041856474574
10 1 1
10 6 6
12 0.5316343278593716
12 3 3
14 0.4271181077665984
14 6 6
26 0.374398228616208
26 1 1
27 0.377571675271481
27 1 1
31 0.3738821000761687
31 1 1
33 0.4999204606718717
33 1 1
35 0.5943899125761685
35 23 32
37 0.5535833329863249
37 1 1
37 6 6
37 20 28
38 0.539777943582028
38 1 1
39 1.0131635636992151
39 19 25
40 0.5671838731474574
40 15 21
41 0.5419568986601528
41 3 3
41 15 21
42 0.5998018235869109
42 4 4
43 0.6164617509550747
43 8 13
44 0.5299604386992156
44 1 1
48 0.4041578263945276
48 1 1
49 0.4234739274687467
49 1 1
51 0.6442007035917934
51 27 46
52 0.7821348161406219
52 1 1
52 6 6
53 1.2003122300810514
53 1 1
53 6 6
56 0.6532911271757778
56 15 21
57 0.6953706712431607
57 1 1
57 6 6
57 20 28
58 0.4178335160917932
58 1 1
58 6 6
58 20 28
59 0.4042860002226529
59 1 1
60 0.5272088975615201
60 23 32
61 0.3819986314482388
61 1 1
63 0.4042516679472623
63 1 1
64 0.46080379

In [55]:
s1 = Structure.from_str(generations_df.at[10, 'StructureRelaxed'], fmt='json')
s2 = Structure.from_str(generations_df.at[6, 'StructureRelaxed'], fmt='json')
s_mp.matches(s2)

True

In [58]:
s_mp = Structure.from_file("visualization/KZnF3.cif")
s_mp.matches(s2)

False